In [1]:
%pip install bert-for-tf2

  Using cached bert_for_tf2-0.14.9-py3-none-any.whl
  Using cached params_flow-0.8.2-py3-none-any.whl
  Using cached py_params-0.10.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install tensorflow-hub

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 코드 10-41 라이브러리 호출 및 데이터셋 준비

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import pandas as pd

movie_reviews = pd.read_csv("data/IMDB Dataset.csv")
# 데이터셋(movie_reviews)에서 어떤 항목이 NaN을 가지고 있는지 확인
movie_reviews.isnull().values.any()
movie_reviews.shape

2024-12-03 03:38:26.759721: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-12-03 03:38:26.759755: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


(50000, 2)

In [3]:
# 코드 10-42 데이터셋 전처리
import re


def preprocess_text(sen):
    sentence = remove_tags(sen)  # html 태그 삭제
    # 구두점(punctuation) 및 숫자(number) 제거, 문자(a~z, A~Z)가 아닌 것 제거
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", " ", sentence)  # 단일 문자 제거(예 a)
    sentence = re.sub(r"\s+", " ", sentence)  # 두 개 이상의 공백 제거
    return sentence


TAG_RE = re.compile(r"<[^>]+>")  # 정규 표현식(<[^>]+>)을 컴파일


def remove_tags(text):
    return TAG_RE.sub("", text)


reviews = []
sentences = list(movie_reviews["review"])
for sen in sentences:
    # 모든 텍스트 리뷰 데이터를 preprocess_text 함수에 적용
    reviews.append(preprocess_text(sen))

print(movie_reviews.columns.values)  # 데이터셋의 열에 대한 이름 반환

['review' 'sentiment']


In [4]:
# 코드 10-43 sentiment 열에 대한 고윳값 확인
movie_reviews.sentiment.unique()

array(['positive', 'negative'], dtype=object)

In [5]:
# 코드 10-44 긍정/부정 감정 변환
import numpy as np
y = movie_reviews["sentiment"]
y = np.array(list(map(lambda x: 1 if x == "positive" else 0, y)))

In [6]:
# 코드 10-45 리뷰 출력
print(reviews[10])

Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines At first it was very odd and pretty funny but as the movie progressed didn find the jokes or oddness funny anymore Its low budget film thats never problem in itself there were some pretty interesting characters but eventually just lost interest imagine this film would appeal to stoner who is currently partaking For something similar but better try Brother from another planet 


In [7]:
# 코드 10-46 긍정/부정 리뷰 확인
print(y[10])

0


In [8]:
# 코드 10-47 텍스트의 토큰화
# bert.bert_tokenization 모듈의 FullTokenizer 클래스를 사용하여 객체를 만듭니다.
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False,
)
# 넘파이 배열 형식의 BERT 어휘 파일을 만듭니다.
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# 텍스트를 소문자로 설정합니다.
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
# vocabulary_file 및 to_lower_case 변수를 BertTokenizer 객체에 전달합니다.
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

2024-12-03 03:38:33.446700: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-12-03 03:38:33.596269: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:968] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-03 03:38:33.596298: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 298.32GiB/s
2024-12-03 03:38:33.596351: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-12-03 03:38:33.596381: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic 

In [9]:
# 코드 10-48 임의의 문장 토큰화
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [10]:
# 코드 10-49 토큰의 ID 반환
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("don't be so judgmental"))

[2123, 1005, 1056, 2022, 2061, 8689, 2389]

In [11]:
# 코드 10-50 리뷰 텍스트 데이터 토큰화
def tokenize_reviews(text_reviews):
    # 단일 텍스트 리뷰를 입력으로 받아들이면 토큰화된 단어의 ID를 반환
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))


# 실제로 입력 데이터셋의 모든 리뷰를 토큰화
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [12]:
# 코드 10-51 문장 길이 통일
import random

# 토큰화된 리뷰, 리뷰 레이블, 리뷰 길이가 포함된 리스트(list)를 생성
reviews_with_len = [
    [review, y[i], len(review)] for i, review in enumerate(tokenized_reviews)
]
random.shuffle(reviews_with_len)
# sort( ) 메서드를 사용하여 리뷰를 기준으로 데이터를 정렬
reviews_with_len.sort(key=lambda x: x[2])
sorted_reviews_labels = [
    (review_lab[0], review_lab[1]) for review_lab in reviews_with_len
]
# sorted_reviews_labels, output_types에 대한 결과를 int32 형식으로 출력
processed_dataset = tf.data.Dataset.from_generator(
    lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32)
)
# 배치 크기를 32로 설정, 즉 리뷰 32건을 처리한 후 신경망의 가중치를 업데이트
BATCH_SIZE = 32
# 데이터셋에 패딩을 적용
batched_dataset = processed_dataset.padded_batch(
    BATCH_SIZE, padded_shapes=((None,), ())
)

next(iter(batched_dataset))  # 첫 번째 배치를 출력하고 패딩이 어떻게 적용되었는지 확인

(<tf.Tensor: shape=(32, 21), dtype=int32, numpy=
 array([[ 2054,  5896,  2054,  2466,  2054,  6752,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [ 3078,  5436,  3078,  3257,  3532,  7613,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [ 3191,  1996,  2338,  5293,  1996,  3185,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [ 2062, 23873,  3993,  2062, 11259,  2172,  2172,  2062, 14888,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [ 2023,  3185,  2003,  6659,  2021,  2009,  2038,  2070,  2204,
          3896,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [ 1045,  2876,  9278,  2023,  2028,  2130,  2006,  7922, 12635,
    

In [13]:
# 코드 10-52 데이터셋을 훈련과 검증 세트로 분리
import math

# 전체 레코드를 32(배치 크기)로 나누어 줌으로써 전체 배치 크기를 구합니다.
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
# 데이터의 10%는 검증을 위해 남겨 둡니다.
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)

# test_data에 데이터를 저장하려고 batched_dataset( ) 객체의 take( ) 메서드를 사용합니다.
test_data = batched_dataset.take(TEST_BATCHES)
# 나머지 데이터는 skip( ) 메서드를 사용하여 훈련을 위해 train_data에 저장합니다.
train_data = batched_dataset.skip(TEST_BATCHES)

In [14]:
class TEXT_MODEL(tf.keras.Model):
    def __init__(
        self,
        vocabulary_file,
        embedding_dimnesions=128,
        cnn_filters=50,
        dnn_units=512,
        model_output_classes=2,
        dropout_rate=0.1,
        training=False,
        name="text_model",
    ):
        super(TEXT_MODEL, self).__init__(name=name)

        self.embedding = tf.keras.layers.Embedding(
            vocabulary_file, embedding_dimnesions
        )
        self.cnn_layer1 = tf.keras.layers.Conv1D(
            filters=cnn_filters, kernel_size=2, padding="valid", activation="relu"
        )
        self.cnn_layer2 = tf.keras.layers.Conv1D(
            filters=cnn_filters, kernel_size=3, padding="valid", activation="relu"
        )
        self.cnn_layer3 = tf.keras.layers.Conv1D(
            filters=cnn_filters, kernel_size=4, padding="valid", activation="relu"
        )
        # 합성곱 신경망 계층 세 개가 각각 커널 또는 필터 값(2, 3, 4)으로 초기화되었습니다. 이때 원한다면 필터 크기를 변경해 볼 수 있습니다.
        self.pool = tf.keras.layers.GlobalMaxPool1D()
        self.dense_1 = tf.keras.layers.Dense(units=dnn_units, activation="relu")
        # 10% 비율에 대해 드롭아웃 (deactivation)을 적용합니다.
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)

        if model_output_classes == 2:
            self.last_dense = tf.keras.layers.Dense(units=1, activation="sigmoid")
        else:
            self.last_dense = tf.keras.layers.Dense(
                units=model_output_classes, activation="softmax"
            )

    # 함수를 호출하는 것처럼 클래스의 객체도 호출할 수 있게 만들 수 있는데, 이때 필요한 메서드가 __call__입니다.
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l)
        l_1 = self.pool(l_1)
        l_2 = self.cnn_layer2(l)
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        # call() 함수 내에서 각 합성곱 신경망 계층의 출력에 전역 최대 풀링을 적용합니다.
        l_3 = self.pool(l_3)

        # 합성곱층 세 개가 함께 연결(concat)되고 그 출력이 첫 번째 신경망에 공급됩니다.
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1)
        # 두 번째로 연결된 신경망은 클래스 두 개만 포함하므로 출력 감정을 예측하는 데 사용됩니다.
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training=training)
        # 모델의 출력층
        model_output = self.last_dense(concatenated)

        return model_output

In [15]:
# 코드 10-54 하이퍼파라미터 초기화
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 5

In [16]:
# 코드 10-55 하이퍼파라미터 값을 네트워크에 전달
text_model = TEXT_MODEL(
    vocabulary_file=VOCAB_LENGTH,
    embedding_dimnesions=EMB_DIM,
    cnn_filters=CNN_FILTERS,
    dnn_units=DNN_UNITS,
    model_output_classes=OUTPUT_CLASSES,
    dropout_rate=DROPOUT_RATE,
)

In [17]:
# 코드 10-56 모델 훈련
if OUTPUT_CLASSES == 2:
    text_model.compile(
        loss="binary_crossentropy",
        optimizer="adam",
        metrics=["accuracy"],
    )
else:
    text_model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"],
    )

text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
1407/1407 [==============================] - 59s 42ms/step - loss: 0.3017 - accuracy: 0.8667
Epoch 2/5
1407/1407 [==============================] - 60s 43ms/step - loss: 0.1333 - accuracy: 0.9521
Epoch 3/5
1407/1407 [==============================] - 60s 43ms/step - loss: 0.0710 - accuracy: 0.9749
Epoch 4/5
1407/1407 [==============================] - 59s 42ms/step - loss: 0.0432 - accuracy: 0.9846
Epoch 5/5
1407/1407 [==============================] - 59s 42ms/step - loss: 0.0221 - accuracy: 0.9923


In [18]:
# 코드 10-57 모델 성능 평가
results = text_model.evaluate(test_data)
print(results)

156/156 [==============================] - 1s 7ms/step - loss: 0.4391 - accuracy: 0.8968
[0.43907997012138367, 0.8968349099159241]


In [19]:
# 코드 10-58 라이브러리 호출
import pandas as pd
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
# 코드 10-59 데이터셋 메모리로 로딩
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

In [ ]:
# %pip install bert-for-tf2

In [21]:
# 코드 10-60 텍스트 토큰화
url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2"
bert_layer = hub.KerasLayer(url, trainable=True)

FullTokenizer = bert.bert_tokenization.FullTokenizer

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [25]:
# 코드 10-61 텍스트 전처리
import numpy as np


def bert_encoder(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in texts:
        text = tokenizer.tokenize(text)  # 입력 데이터를 토큰으로 변환
        text = text[: max_len - 2]
        # CLS/SEP 처리(버트에서 입력 값에 대한 임베딩을 위한 식별자)
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        # 제로 패딩 적용
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


train_input = bert_encoder(train_data, tokenizer, max_len=160)
train_labels = train_data.target.values

In [26]:
# 코드 10-62 모델 생성
def build_model(max_len=512):
    # 입력층에서 사용할 input_word_ids 정의
    input_word_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="input_word_ids"
    )
    # 입력층에서 사용할 input_segment_ids 정의
    input_segment_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="input_segment_ids"
    )
    # 입력층에서 사용할 input_mask 정의
    input_mask = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="input_mask"
    )
    # 버트 계층의 입력 값으로 앞서 정의한 input_word_ids, input_segment_ids, input_mask 사용
    pooled_output, sequence_output = bert_layer(
        [input_word_ids, input_mask, input_segment_ids]
    )
    clf_output = pooled_output
    # 출력층(완전연결층) 정의
    output = tf.keras.layers.Dense(1, activation="sigmoid")(clf_output)
    model = tf.keras.Model(
        inputs=[input_word_ids, input_mask, input_segment_ids], outputs=output
    )
    model.compile(
        optimizer=RMSprop(learning_rate=2e-6), loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

model = build_model(max_len=160)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_segment_ids (InputLayer)  [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [ ]:
# 코드 10-63 모델 훈련
train_history = model.fit(
    train_input, train_labels, validation_split=0.2, epochs=3, batch_size=8
)

Epoch 1/3
1/1 [==============================] - 1s 1s/step - loss: 0.3820 - accuracy: 0.7500 - val_loss: 0.0614 - val_accuracy: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 341ms/step - loss: 0.0996 - accuracy: 1.0000 - val_loss: 0.0504 - val_accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 348ms/step - loss: 0.1760 - accuracy: 1.0000 - val_loss: 0.0485 - val_accuracy: 1.0000


: 